In [ ]:
import tensorflow as tf
import os
import cv2
import imghdr

data_dir="your directory here"

os.listdir(data_dir)

img_exts=['jpeg','jpg','bmp','png']

for image_class in os.listdir(data_dir):
    for image in os.listdir(os.path.join(data_dir,image_class)):
        image_path=os.path.join(data_dir,image_class,image)
        try:
            img=cv2.imread(image_path)
            tip=imghdr.what(image_path)
            if tip not in img_exts:
                print("Image not in ext list{}".format(image_path))
                os.remove(image_path)
        except Exception as e:
            print("Issue with Iamge{}".format(image_path))

import numpy as np
from matplotlib import pyplot as plt

data=tf.keras.utils.image_dataset_from_directory("your directory here")

data_iterator=data.as_numpy_iterator()

batch=data_iterator.next()

fig, ax=plt.subplots(ncols=4,figsize=(20,20))
for idx,img in enumerate(batch[0][:4]):
    ax[idx].imshow(img.astype(int))
    ax[idx].title.set_text(batch[1][idx])

data=data.map(lambda x,y:(x/255,y))

train_size=int(len(data)*.7)
val_size=int(len(data)*.2)
test_size=int(len(data)*.1)

train=data.take(train_size)
val=data.skip(train_size).take(val_size)
test=data.skip(train_size+val_size).take(test_size)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout

model=Sequential()

model.add(Conv2D(16,(3,3),1,activation='relu',input_shape=(256,256,3)))
model.add(MaxPooling2D())

model.add(Conv2D(32,(3,3),1,activation='relu'))
model.add(MaxPooling2D())

model.add(Conv2D(16,(3,3),1,activation='relu'))
model.add(MaxPooling2D())

model.add(Flatten())

model.add(Dense(256,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

model.compile('adam',loss=tf.losses.BinaryCrossentropy(),metrics=['accuracy'])

model.summary()

logdir="your directory here"

t_callback=tf.keras.callbacks.TensorBoard(log_dir=logdir)

hist=model.fit(train,epochs=20,validation_data=val,callbacks=[t_callback])

fig=plt.figure()
plt.plot(hist.history['loss'],color='teal',label='loss')
plt.plot(hist.history['val_loss'],color='orange',label='val_loss')
fig.suptitle('Loss',fontsize=20)
plt.legend(loc='upper right')
plt.show()

fig=plt.figure()
plt.plot(hist.history['accuracy'],color='teal',label='accuracy')
plt.plot(hist.history['val_accuracy'],color='orange',label='val_accuracy')
fig.suptitle('Accuracy',fontsize=20)
plt.legend(loc='lower right')
plt.show()

from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy

pre=Precision()
re=Recall()
acc=BinaryAccuracy()

for batch in test.as_numpy_iterator():
    x,y=batch
    yhat=model.predict(x)
    pre.update_state(y,yhat)
    re.update_state(y,yhat)
    acc.update_state(y,yhat)

print(pre.result().numpy(),re.result().numpy(),acc.result().numpy())

img=cv2.imread("your directory here")
plt.imshow(cv2.cvtColor(img,cv2.COLOR_BGR2RGB))
plt.show()

resize=tf.image.resize(img,(256,256))
plt.imshow(resize.numpy().astype(int))
plt.show()

yhat=model.predict(np.expand_dims(resize/255,0))

yhat

if yhat> 0.5:
    print("Image is Sad")
else:
    print("Image is Happy")

imgg=cv2.imread("your directory here")
plt.imshow(cv2.cvtColor(imgg,cv2.COLOR_BGR2RGB))
plt.show()

resize=tf.image.resize(imgg,(256,256))
plt.imshow(resize.numpy().astype(int))
plt.show()

yhatt=model.predict(np.expand_dims(resize/255,0))

yhatt

if yhatt> 0.5:
    print("Image is Sad")
else:
    print("Image is Happy")